## One-way ANOVA Testing for Time Completion on Task 1 (Urgent Task)

In [1]:
import pandas as pd
from scipy.stats import f_oneway

In [3]:
sample_df_orig = pd.read_csv("data1.csv")

def extract_numerical(value):
    return float(value.split()[0])
    
sample_df = sample_df_orig.apply(lambda x: x.map(extract_numerical))

sample_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42 entries, 0 to 41
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   DLTask1  42 non-null     float64
 1   DCTask1  42 non-null     float64
 2   RLTask1  42 non-null     float64
 3   RCTask1  42 non-null     float64
 4   CTask1   42 non-null     float64
dtypes: float64(5)
memory usage: 1.8 KB


In [4]:
print(sample_df)

     DLTask1   DCTask1  RLTask1  RCTask1   CTask1
0     4585.0    9220.0   1481.0   6057.0   6829.0
1     1247.0    9587.0   4507.0   7369.0  18039.0
2   112068.0    4566.0   4184.0   5554.0   3156.0
3     6991.0   18855.0   4483.0   5954.0   5923.0
4     8818.0   24423.0   4818.0  12067.0  11761.0
5    14566.0    6751.0   1830.0  12981.0  15037.0
6    22405.0  195647.0   5024.0   4783.0  13268.0
7    13782.0    8398.0   3128.0   5227.0   2326.0
8     2241.0    5256.0   9960.0   6453.0   3642.0
9   159866.0   34348.0  12233.0  10260.0   8386.0
10   64355.0   32844.0   4830.0  11648.0   5572.0
11   12813.0    3599.0   2853.0   7591.0   3839.0
12    3279.0    7591.0   4308.0   3648.0   6375.0
13   11064.0   14553.0   7052.0  14012.0  11616.0
14    5316.0    5957.0   6741.0  10426.0  14824.0
15   10351.0   16871.0   4334.0  16143.0  15174.0
16    7204.0    4787.0   6250.0  10700.0   6339.0
17  201185.0   36243.0   6827.0   7735.0   6161.0
18   51583.0   92190.0  21895.0  15148.0   9107.0


In [5]:
#Get the mean first


means = sample_df.mean()

print("Mean of each column:")
print(means)

Mean of each column:
DLTask1    28963.785714
DCTask1    18664.285714
RLTask1     5463.380952
RCTask1     7699.309524
CTask1      6180.523810
dtype: float64


In [6]:
#Get significant difference using One-way ANOVA
df = sample_df
f_statistic, p_value = f_oneway(df['DLTask1'], df['DCTask1'], df['RLTask1'], df['RCTask1'], df['CTask1'])

# Print results
print("F-Statistic:", f_statistic)
print("P-value:", p_value)

F-Statistic: 7.917095000695259
P-value: 5.9315659027920345e-06


In [20]:
if p_value < 0.05:
    print("There is a significant difference in the time users take to complete the task of clicking the most urgent to-do task across different layouts of the Canvas Dashboard.")
else:
    print("There is a significant difference in the time users take to complete the task of clicking the most urgent to-do task across different layouts of the Canvas Dashboard.")

There is a significant difference in the time users take to complete the task of clicking the most urgent to-do task across different layouts of the Canvas Dashboard.


## One-way ANOVA Testing for Time Completion on Task 2 (Announcement Task)

In [9]:
import pandas as pd
from scipy.stats import f_oneway

In [10]:
sample_df_orig = pd.read_csv("data2.csv")

def extract_numerical(value):
    return float(value.split()[0])
    
sample_df = sample_df_orig.apply(lambda x: x.map(extract_numerical))

sample_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42 entries, 0 to 41
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   DLTask3  42 non-null     float64
 1   DCTask3  42 non-null     float64
 2   RLTask3  42 non-null     float64
 3   RCTask3  42 non-null     float64
 4   CTask3   42 non-null     float64
dtypes: float64(5)
memory usage: 1.8 KB


In [11]:
print(sample_df)

    DLTask3  DCTask3  RLTask3  RCTask3   CTask3
0   16897.0   6042.0   2472.0   8576.0   7511.0
1   58861.0  20932.0   2237.0  12174.0   8445.0
2   56264.0   5610.0  15891.0   2766.0    763.0
3   13081.0  14064.0   4724.0  21063.0   3030.0
4   15987.0   7674.0   6112.0  16130.0   7379.0
5   11029.0   9789.0   3916.0   4979.0   3233.0
6   25834.0   5111.0   4129.0   9485.0  10297.0
7   11393.0  11310.0   1628.0   7878.0   1462.0
8   15899.0  11043.0   8319.0  15561.0   4640.0
9   42390.0   9286.0  21264.0  27040.0   2434.0
10  59390.0  17170.0   2427.0   6136.0   5273.0
11   9871.0   9159.0   2741.0  11472.0   1927.0
12  15961.0   3928.0   2079.0   9496.0   4135.0
13  12924.0  20639.0   7328.0  34083.0   6829.0
14  13257.0   7595.0   3993.0  10354.0   8500.0
15  15697.0  16963.0   5826.0   9860.0   1255.0
16  23274.0   8105.0   2644.0  10250.0   4515.0
17   8884.0   3807.0   2194.0   7448.0   6518.0
18  86041.0  12734.0   8091.0  17348.0   7104.0
19  11572.0   4065.0   7213.0  10479.0  

In [12]:
#Get the mean first


means = sample_df.mean()

print("Mean of each column:")
print(means)

Mean of each column:
DLTask3    20010.857143
DCTask3     9298.095238
RLTask3     4414.333333
RCTask3     8416.571429
CTask3      4469.190476
dtype: float64


In [13]:
#Get significant difference using One-way ANOVA
df = sample_df
f_statistic, p_value = f_oneway(df['DLTask3'], df['DCTask3'], df['RLTask3'], df['RCTask3'], df['CTask3'])

# Print results
print("F-Statistic:", f_statistic)
print("P-value:", p_value)

F-Statistic: 17.442879708577834
P-value: 2.465890933317309e-12


In [21]:
if p_value < 0.05:
    print("There is a significant difference in the time users take to complete the task of clicking the most recent unread announcement across different layouts of the Canvas Dashboard.")
else:
    print("There is no significant difference in the time users take to complete the task of clicking the most recent unread announcement across different layouts of the Canvas Dashboard.")

There is a significant difference in the time users take to complete the task of clicking the most recent unread announcement across different layouts of the Canvas Dashboard.


## One-way ANOVA Testing for Idenitifaction Accuracy on Task 1 (Urgent Task)

In [15]:
sample_df_orig = pd.read_csv("data3.csv")

def extract_numerical(value):
    return float(value.split()[0])
    
sample_df = sample_df_orig

sample_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42 entries, 0 to 41
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   DLTask1  42 non-null     float64
 1   DCTask1  42 non-null     float64
 2   RLTask1  42 non-null     float64
 3   RCTask1  42 non-null     float64
 4   CTask1   42 non-null     float64
dtypes: float64(5)
memory usage: 1.8 KB


In [16]:
print(sample_df)

     DLTask1   DCTask1   RLTask1   RCTask1    CTask1
0   0.500000  0.500000  1.000000  1.000000  1.000000
1   0.071429  0.333333  1.000000  1.000000  1.000000
2   1.000000  0.500000  1.000000  0.333333  1.000000
3   0.250000  0.500000  1.000000  1.000000  1.000000
4   0.100000  1.000000  1.000000  1.000000  1.000000
5   0.500000  1.000000  1.000000  1.000000  1.000000
6   1.000000  1.000000  1.000000  1.000000  1.000000
7   0.333333  0.200000  1.000000  1.000000  1.000000
8   1.000000  0.500000  1.000000  1.000000  1.000000
9   0.142857  1.000000  1.000000  1.000000  1.000000
10  1.000000  1.000000  1.000000  1.000000  1.000000
11  1.000000  1.000000  0.333333  1.000000  1.000000
12  0.250000  0.250000  1.000000  1.000000  0.500000
13  0.111111  0.500000  1.000000  1.000000  1.000000
14  0.200000  0.500000  1.000000  0.200000  1.000000
15  0.142857  0.250000  0.500000  0.027778  1.000000
16  1.000000  0.500000  0.250000  0.500000  1.000000
17  0.333333  0.333333  0.500000  1.000000  1.

In [17]:
#Get the mean first


means = sample_df.mean()

print("Mean of each column:")
print(means)

Mean of each column:
DLTask1    0.483897
DCTask1    0.625286
RLTask1    0.857143
RCTask1    0.872487
CTask1     0.948413
dtype: float64


In [18]:
#Get significant difference using One-way ANOVA
df = sample_df
f_statistic, p_value = f_oneway(df['DLTask1'], df['DCTask1'], df['RLTask1'], df['RCTask1'], df['CTask1'])

# Print results
print("F-Statistic:", f_statistic)
print("P-value:", p_value)

F-Statistic: 17.258661732706678
P-value: 3.222526390638752e-12


In [22]:
if p_value < 0.05:
    print("There is a significant difference in the identification accuracy when completing the task of clicking the most urgent to-do task across different layouts of the Canvas Dashboard.")
else:
    print("There is no significant difference in the identification accuracy when completing the task of clicking the most urgent to-do task across different layouts of the Canvas Dashboard.")

There is a significant difference in the identification accuracy when completing the task of clicking the most urgent to-do task across different layouts of the Canvas Dashboard.


## One-way ANOVA Testing for Idenitifaction Accuracy on Task 2 (Announcement Task)

In [25]:
sample_df_orig = pd.read_csv("data4.csv")

def extract_numerical(value):
    return float(value.split()[0])
    
sample_df = sample_df_orig

sample_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42 entries, 0 to 41
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   DLTask2  42 non-null     float64
 1   DCTask2  42 non-null     float64
 2   RLTask2  42 non-null     float64
 3   RCTask2  42 non-null     float64
 4   CTask2   42 non-null     float64
dtypes: float64(5)
memory usage: 1.8 KB


In [26]:
print(sample_df)

     DLTask2   DCTask2   RLTask2   RCTask2    CTask2
0   0.250000  0.250000  1.000000  0.500000  1.000000
1   0.142857  0.500000  1.000000  1.000000  1.000000
2   0.055556  0.071429  1.000000  0.250000  1.000000
3   0.333333  1.000000  1.000000  0.500000  1.000000
4   0.125000  0.500000  0.500000  1.000000  1.000000
5   1.000000  1.000000  1.000000  1.000000  1.000000
6   0.250000  1.000000  1.000000  0.100000  1.000000
7   1.000000  1.000000  1.000000  0.333333  1.000000
8   0.500000  1.000000  1.000000  1.000000  1.000000
9   1.000000  1.000000  1.000000  1.000000  1.000000
10  0.500000  0.166667  1.000000  0.500000  1.000000
11  1.000000  0.500000  1.000000  1.000000  1.000000
12  0.076923  0.500000  1.000000  1.000000  1.000000
13  0.035714  1.000000  1.000000  1.000000  1.000000
14  0.111111  0.166667  1.000000  0.142857  1.000000
15  1.000000  0.250000  1.000000  1.000000  0.500000
16  0.111111  0.250000  0.250000  0.100000  1.000000
17  0.333333  0.500000  1.000000  0.200000  1.

In [27]:
#Get the mean first


means = sample_df.mean()

print("Mean of each column:")
print(means)

Mean of each column:
DLTask2    0.503618
DCTask2    0.647336
RLTask2    0.923413
RCTask2    0.706444
CTask2     0.928571
dtype: float64


In [28]:
#Get significant difference using One-way ANOVA
df = sample_df
f_statistic, p_value = f_oneway(df['DLTask2'], df['DCTask2'], df['RLTask2'], df['RCTask2'], df['CTask2'])

# Print results
print("F-Statistic:", f_statistic)
print("P-value:", p_value)

F-Statistic: 14.544778236868032
P-value: 1.7905875882147234e-10


In [29]:
if p_value < 0.05:
    print("There is a significant difference in the identification accuracy when completing the task of clicking the most recent unread announcement across different layouts of the Canvas Dashboard.")
else:
    print("There is no significant difference in the identification accuracy when completing the task of clicking the most recent unread announcement across different layouts of the Canvas Dashboard.")

There is a significant difference in the identification accuracy when completing the task of clicking the most recent unread announcement across different layouts of the Canvas Dashboard.
